In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)

# Define leg parameters
coxa_len = 28.75
tibia_len = 68.825
femur_len = 40
z_offset = tibia_len
l_len = np.sqrt( z_offset**2 + coxa_len**2 ) # Z-offset can change!

def to_rads(num):
    return num*np.pi/180

def to_degs(num):
    return num/np.pi*180

def pythagoras(nums):
    nums = np.power(nums, 2)
    sums = np.sum(nums)
    # print(order, nums, sums)

    return np.sqrt(sums)

In [68]:
# Forward kinematics check - make this into a function
def plot_legs(angle):
    leg_base = np.array([0, 0, z_offset], dtype='float32') # xyz
    coxa_end = np.array([0, 0, z_offset], dtype='float32')
    femur_end = np.array([0, 0, 0], dtype='float32')
    tibia_end = np.array([0, 0, 0], dtype='float32')

    # Input desired angles here
    print("\nDesired angles:", ["%0.4f" %i for i in angle])
    angles = angle
    coxa_angle = angles[0] - 90
    femur_angle = angles[1] - 90
    tibia_angle = angles[2]

    theta = tibia_angle + femur_angle - 90

    femur_vert = femur_len * np.cos( to_rads(femur_angle) )
    tibia_vert = tibia_len * np.sin( to_rads(theta) )

    femur_hor = femur_len * np.cos( to_rads(coxa_angle) )
    tibia_hor = tibia_len * np.cos( to_rads(coxa_angle) )

    # # Debug parameters
    # print(z_offset)
    # # Debug desired angles
    # print("%0.4f, %0.4f, %0.4f, %0.4f" %(coxa_angle, femur_angle, tibia_angle, theta))
    # # debug tibia params
    # print("%0.4f, %0.4f" %(femur_vert, tibia_vert))

    # calc coxa end-points
    coxa_end[0] = coxa_len * np.sin( to_rads(coxa_angle) )
    coxa_end[1] = coxa_len * np.cos( to_rads(coxa_angle) )

    # calc femur end-points
    femur_end[0] = coxa_end[0] + femur_vert * np.sin( to_rads(coxa_angle) )
    femur_end[1] = coxa_end[1] + femur_vert * np.cos( to_rads(coxa_angle) )
    femur_end[2] = coxa_end[2] + femur_hor * np.sin( to_rads(femur_angle) )

    # calc tibia end-points
    tibia_end[0] = femur_end[0] + tibia_vert * np.sin( to_rads(coxa_angle) )
    tibia_end[1] = femur_end[1] + tibia_vert * np.cos( to_rads(coxa_angle) )
    tibia_end[2] = femur_end[2] + tibia_hor * np.cos( to_rads( theta-180 ) )

    # sanity check: if all legs are the same length
    print("Coxa length:", pythagoras(coxa_end-leg_base))
    print("Femur length:", pythagoras(femur_end-coxa_end))
    print("Tibia length:", pythagoras(tibia_end-femur_end))

    print("Leg_base:", ["%0.4f" %i for i in leg_base])
    print("coxa_end:", ["%0.4f" %i for i in coxa_end])
    print("femur_end:", ["%0.4f" %i for i in femur_end])
    print("tibia_end:", ["%0.4f" %i for i in tibia_end])
    print("theta: %0.4f" %theta)

    # TODO print end coords onto visualization;
    # Plot both views
    fig, axs = plt.subplots(1,2)
    fig.suptitle("Leg IK simulation")
    axs[0].set_title("Top View")
    axs[0].set_xlabel("y")
    axs[0].set_xlabel("x")
    axs[0].plot( (leg_base[1], coxa_end[1]), (leg_base[0], coxa_end[0]), 'x-', linewidth=5, label="Coxa")
    axs[0].plot( (coxa_end[1], femur_end[1]), (coxa_end[0], femur_end[0]), 'x-', linewidth=5, label="Femur")
    axs[0].plot( (femur_end[1], tibia_end[1]), (femur_end[0], tibia_end[0]), 'x-', linewidth=5, label="Tibia")
    axs[0].set_ylim(-45, 45)
    axs[0].set_xlim(0, 90)
    axs[0].grid()

    axs[1].set_title("Side view")
    axs[1].set_xlabel("y")  
    axs[1].set_xlabel("x")
    axs[1].plot( (leg_base[1], coxa_end[1]), (leg_base[2], coxa_end[2]), 'x-', linewidth=5, label="Coxa")
    axs[1].plot( (coxa_end[1], femur_end[1]), (coxa_end[2], femur_end[2]), 'x-', linewidth=5, label="Femur")
    axs[1].plot( (femur_end[1], tibia_end[1]), (femur_end[2], tibia_end[2]), 'x-', linewidth=5, label="Tibia")
    axs[1].set_ylim(-30, 90)
    axs[1].set_xlim(0, 120)
    axs[1].grid()

    plt.legend()
    plt.tight_layout()
    plt.show()

    return [leg_base, coxa_end, femur_end, tibia_end, theta]

In [73]:
# inverse kinematics check (frame this WRT leg origin)
# There is some error still when x is isolated. Where does it come from?

leg_end_coords = [0.0, 68.75, 0.0] # check back

# leg_end_coords[1] -= 68.75 # remove offsets
coxa_angle_ik = np.arctan2(leg_end_coords[0], leg_end_coords[1]) # 'gamma' in notes

coxa_len_ik = coxa_len * np.cos( coxa_angle_ik ) # Takes side view of this thing.
femur_len_ik = femur_len * np.cos( coxa_angle_ik )
tibia_len_ik = tibia_len * np.cos( coxa_angle_ik )

y_offset_ik = leg_end_coords[1] - coxa_len_ik
z_offset_ik = z_offset - leg_end_coords[2]
l_len = np.sqrt( z_offset_ik**2 + y_offset_ik**2 )

print("z_offset_ik: %0.4f | y_offset_ik: %0.4f | l_len: %0.4f" %(z_offset_ik, y_offset_ik, l_len) )

femur_angle_ik1 = np.arccos( z_offset_ik / l_len )
# print(z_offset_ik / l_len)
femur_angle_ik2 = np.arccos( (femur_len_ik**2 + l_len**2 - tibia_len_ik**2) / (2*femur_len_ik*l_len) )
# print((femur_len**2 + l_len**2 - tibia_len**2) / (2*femur_len*l_len))
# print(to_degs(femur_angle_ik1), to_degs(femur_angle_ik2))
femur_angle_ik = femur_angle_ik1 + femur_angle_ik2

tibia_angle_ik = np.arccos( (femur_len_ik**2 - l_len**2 + tibia_len_ik**2) / (2*femur_len_ik*tibia_len_ik) )
# print((femur_len**2 - l_len**2 + tibia_len**2) / (2*femur_len*tibia_len))

angles_ik = [to_degs(coxa_angle_ik)+90, to_degs(femur_angle_ik), to_degs(tibia_angle_ik)]

cool = plot_legs(angles_ik)

z_offset_ik: 68.8250 | y_offset_ik: 40.0000 | l_len: 79.6045

Desired angles: ['90.0000', '90.0000', '90.0000']
Coxa length: 28.75
Femur length: 40.0
Tibia length: 68.825
Leg_base: ['0.0000', '0.0000', '68.8250']
coxa_end: ['0.0000', '28.7500', '68.8250']
femur_end: ['0.0000', '68.7500', '68.8250']
tibia_end: ['0.0000', '68.7500', '-0.0000']
theta: 0.0000
